<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [9]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [5]:
!pip install gensim

  Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl (24.0 MB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [26]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
import numpy as np
from scipy.linalg import norm

def cosine_sim(v1, v2):
    # v1, v2 (1 x dim)
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

# Метод для поиска наиболее похожих векторов
def most_similar(vector, topn=10):
    similarities = []
    for word in wv_embeddings.index_to_key:
        word_vector = wv_embeddings[word]
        sim = cosine_sim(vector, word_vector)
        similarities.append((word, sim))
    
    # Сортируем по убыванию сходства
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Возвращаем topn наиболее похожих слов
    return similarities[:topn]

Найдем наиболее близкие слова к слову `dog`:


In [34]:
query_vector = wv_embeddings["dog"]  
similar_words = most_similar(query_vector, topn=5)
print(similar_words)

[('dog', 1.0000000828170434), ('animal', 0.8564179513718673), ('dogs', 0.7880866310970126), ('mammal', 0.7623804437080305), ('cats', 0.7621252774580861)]


#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [46]:
# method most_simmilar
similar_words = most_similar(query_vector, topn=40)

In [47]:
import pprint
pprint.pprint(similar_words)

[('dog', 1.0000000828170434),
 ('animal', 0.8564179513718673),
 ('dogs', 0.7880866310970126),
 ('mammal', 0.7623804437080305),
 ('cats', 0.7621252774580861),
 ('animals', 0.7607938048034635),
 ('feline', 0.7392398042479146),
 ('bird', 0.7315488806624892),
 ('animal1', 0.7219215607167601),
 ('doggy', 0.7213349022463522),
 ('labrador', 0.7209132095834343),
 ('canine', 0.7209056863687269),
 ('meow', 0.7185295698758795),
 ('cow', 0.7080444802043843),
 ('dog2', 0.7057910999533431),
 ('woof', 0.7050612363361479),
 ('dog1', 0.7038840554374048),
 ('dog3', 0.7018820123727572),
 ('penguin', 0.6970292625071599),
 ('bulldog', 0.6940488762972771),
 ('mammals', 0.6931389049701139),
 ('bark', 0.6913798450996422),
 ('fruit', 0.6892251362669571),
 ('reptile', 0.6891210143412323),
 ('furry', 0.6863499230457885),
 ('carnivore', 0.6862948919939301),
 ('cat', 0.6852341698838184),
 ('horse', 0.683338188527309),
 ('kitten', 0.6820153744181429),
 ('sheep', 0.6802570806228189),
 ('chihuahua', 0.679175771180810

In [49]:
for i, (word, _) in enumerate(similar_words, 1):
    if word == 'cat':
        cat_position = i
        break
print(i)

27


***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'. Занимает 27 место

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [13]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [14]:

def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    Преобразует вопрос в векторное представление как среднее векторов всех слов в вопросе.

    Параметры:
        question: строка (вопрос)
        embeddings: объект с векторными представлениями слов (например, KeyedVectors)
        tokenizer: объект для токенизации текста
        dim: размерность вектора (по умолчанию 200)

    Возвращает:
        Векторное представление вопроса (numpy array размерности dim)
    """
    # Токенизируем вопрос
    words = tokenizer.tokenize(question)
    
    # Инициализируем нулевой вектор
    vec = np.zeros(dim)
    
    # Считаем количество слов, для которых есть векторное представление
    valid_words_count = 0
    
    # Проходим по всем словам в вопросе
    for word in words:
        if word in embeddings:  # Проверяем, есть ли слово в embeddings
            vec += embeddings[word]  # Добавляем вектор слова
            valid_words_count += 1
    
    # Если есть хотя бы одно слово с вектором, возвращаем среднее
    if valid_words_count > 0:
        vec /= valid_words_count
    
    return vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [57]:
# Предложение
question = "I love neural networks"

question_vector = question_to_vec(question, wv_embeddings, tokenizer)
print(round(question_vector[2],2))

-1.29


***Ваш ответ:*** -1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)


***Ваш ответ:*** DCG@10≈0.3


#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



Максимум 
Hits@47
−
DCG@1
Hits@47−DCG@1 равен 1.

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [15]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k

    if not isinstance(dup_ranks, list):
        raise ValueError("dup_ranks должен быть списком")
    if not isinstance(k, int) or k < 0:
        raise ValueError("k должен быть неотрицательным целым числом")
    if not dup_ranks:  # Если список пуст
        return 0

    # Подсчет количества дубликатов, чей ранг <= k
    hits_value = 0
    for rank in dup_ranks:
        if not isinstance(rank, int) or rank < 1:
            raise ValueError("Ранги в dup_ranks должны быть положительными целыми числами")
        if rank <= k:
            hits_value += 1

    return hits_value

In [17]:
dup_ranks = [4]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0
Hits@4 = 1


In [12]:
dup_ranks = [1]
k = 10
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

Hits@1 = 1


In [ ]:


import math

def dcg_score(dup_ranks, k):
    """
    Вычисляет DCG@k для заданного списка рангов дубликатов.

    Параметры:
        dup_ranks: список индексов дубликатов (их позиции в ранжированном списке)
        k: пороговое значение для ранга

    Возвращает:
        dcg_value: значение DCG@k
    """
    dcg_value = 0.0

    # Проходим по всем позициям от 1 до k
    for i in range(1, k + 1):
        if i in dup_ranks:  # Если позиция i есть в списке дубликатов
            rel_i = 1  # Релевантность = 1
        else:
            rel_i = 0  # Релевантность = 0

        # Добавляем к DCG
        dcg_value += rel_i / math.log2(i + 1)

    return dcg_value

In [19]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [17]:
import pandas as pd

In [22]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [1]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [2]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [25]:
len(validation_data)

3760

Размер нескольких первых строк

In [3]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [18]:

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
    Ранжирует кандидатов на основе косинусного расстояния.

    Параметры:
        question: строка (вопрос)
        candidates: список строк (кандидаты)
        embeddings: объект с векторными представлениями слов (например, KeyedVectors)
        tokenizer: объект для токенизации текста
        dim: размерность вектора (по умолчанию 200)

    Возвращает:
        result: список пар (начальная позиция, кандидат), отсортированный по убыванию сходства
    """
    # Преобразуем вопрос в вектор
    question_vector = question_to_vec(question, embeddings, tokenizer, dim)

    # Преобразуем кандидатов в векторы
    candidate_vectors = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    # Вычисляем косинусное сходство между вопросом и каждым кандидатом
    similarities = [cosine_similarity([question_vector], [candidate_vector])[0][0] 
                    for candidate_vector in candidate_vectors]

    # Создаем список пар (начальная позиция, кандидат, сходство)
    ranked_candidates = list(zip(range(len(candidates)), candidates, similarities))

    # Сортируем по убыванию сходства
    ranked_candidates.sort(key=lambda x: x[2], reverse=True)

    # Убираем сходство из результата и возвращаем пары (начальная позиция, кандидат)
    result = [(index, candidate) for index, candidate, _ in ranked_candidates]

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [73]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]


In [76]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


`для эксперимента 2`  0, 2, 1.

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [87]:
from tqdm.notebook import tqdm

In [88]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [89]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(validation_data):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [91]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 1.000 | Hits@   1: 285.000
DCG@   5: 2.948 | Hits@   5: 393.000
DCG@  10: 4.544 | Hits@  10: 449.000
DCG@ 100: 19.274 | Hits@ 100: 679.000
DCG@ 500: 39.544 | Hits@ 500: 879.000
DCG@1000: 50.319 | Hits@1000: 1000.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [6]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [7]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [ ]:
# Создаем общий корпус текстов
corpus = []

# Итерируемся по train_data
for item in train_data:
    question = item[0]  
    candidates = item[1]  
    
    # Для каждого кандидата создаем склеенную строку
    for candidate in candidates:
        # Склеиваем вопрос и кандидата
        combined_text = f"{question} {candidate}"
        
        # Токенизируем склеенную строку (в данном случае простой split)
        tokens = combined_text.lower().split()
        
        # Добавляем токенизированный список в корпус
        corpus.append(tokens)

In [11]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=5,                # Размер окна контекста
    min_count=1,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [21]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate((validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [23]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 1.000 | Hits@   1: 157.000
DCG@   5: 2.948 | Hits@   5: 246.000
DCG@  10: 4.544 | Hits@  10: 293.000
DCG@ 100: 18.459 | Hits@ 100: 490.000
DCG@ 500: 44.470 | Hits@ 500: 787.000
DCG@1000: 63.059 | Hits@1000: 1000.000


При k=1: 15.7% правильных ответов на первом месте
При k=5: 24.6% правильных ответов в топ-5
При k=10: 29.3% правильных ответов в топ-10
При k=100: 49% правильных ответов в топ-100
При k=500: 78.7% правильных ответов в топ-500
При k=1000: 100% правильных ответов

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Эффективность эмбеддингов в задачах обработки естественного языка зависит от многих факторов. Морфологическая токенизация с лемматизацией позволяет существенно улучшить качество векторных представлений слов. Нормализация текста играет ключевую роль в подготовке данных, уменьшая шум и фокусируясь на содержательных элементах. Transformer-based модели, такие как BERT, демонстрируют превосходство над классическими подходами благодаря способности учитывать контекст и многозначность слов. Низкое качество решения часто обусловлено недостаточным объемом данных, неэффективной предобработкой и неудачным выбором модели эмбеддингов. Комплексный подход, включающий тщательную токенизацию, использование современных языковых моделей и аугментацию данных, позволяет значительно повысить точность embeddings. Экспериментирование с различными техниками и постоянная оптимизация - ключ к успешной реализации задач обработки естественного языка. В конечном итоге, выбор правильной стратегии создания эмбеддингов определяет эффективность всей системы машинного обучения.